ENVIRON

In [1]:
import networkx as nx
import pandas as pd
import os
import numpy as np
import seaborn as sns
from tqdm import tqdm, trange
from itertools import combinations

/usr/local/lib/python3.8/dist-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
tqdm.pandas()

In [3]:
def flatten(container):
    for i in container:
        if isinstance(i, (list,tuple)):
            for j in flatten(i):
                yield j
        else:
            yield i

In [4]:
for index, model_option in enumerate(os.listdir('./datasets/')):
    # INDEXING
    print(f'[{index}/{len(os.listdir("./datasets/"))}] - {model_option}')
    if model_option == 'original':
        continue

    # DATASET - ROBLOX
    roblox1_df = pd.read_csv(f'./datasets/{model_option}/roblox1.csv', index_col = 0, low_memory = False)
    roblox2_df = pd.read_csv(f'./datasets/{model_option}/roblox2.csv', index_col = 0, low_memory = False)
    roblox3_df = pd.read_csv(f'./datasets/{model_option}/roblox3.csv', index_col = 0, low_memory = False)
    roblox4_df = pd.read_csv(f'./datasets/{model_option}/roblox4.csv', index_col = 0, low_memory = False)
    roblox5_df = pd.read_csv(f'./datasets/{model_option}/roblox5.csv', index_col = 0, low_memory = False)

    df = pd.concat([roblox1_df, roblox2_df, roblox3_df, roblox4_df, roblox5_df]).dropna(subset = ['keybert_keywords']).reset_index(drop = True)

    # PREPROCESS
    tqdm.pandas(desc = '\t(01/10)')
    df['keybert_keywords'] = df['keybert_keywords'].progress_apply(eval)

    tqdm.pandas(desc = '\t(02/10)')
    df['keywords'] = df['keybert_keywords'].progress_apply(lambda x : list(flatten(x))[0::2] if len(x) > 0 else '')

    count_df = df[['keywords', 'userName']].explode('keywords').groupby('keywords').count()

    count_df.columns = ['count']

    # GRAPH FORMULATION - ROBLOX
    G = nx.MultiGraph()
    unique_items = df['keywords'].explode().unique()

    # ADD NODES
    for item in tqdm(unique_items, desc = '\t(03/10)'):
        if count_df.loc[item]['count'] > np.percentile(count_df['count'], 99):
            G.add_node(item, keyword = item, weight = count_df.loc[item]['count'])
    
    selected = list(count_df[count_df['count'] > np.percentile(count_df['count'], 99)].index)
    selected = [word for word in selected if word.isalpha()]

    # ADD EDGES
    for keywords in tqdm(df['keywords'], desc = '\t(04/10)'):
        keyword_selected = [keyword for keyword in keywords if keyword in selected]

        pairs = list(combinations(keyword_selected, 2))
        pairs = [(pair[0], pair[1]) if pair[0] < pair[1] else (pair[1], pair[0]) for pair in pairs]
        
        G.add_edges_from(pairs)
    
    # MULTI to SINGLE - ROBLOX
    H = nx.Graph()
    unique_items = df['keywords'].explode().unique()

    H.add_nodes_from([node for node in G.nodes(data = True) if node[1]['keyword'] in selected])

    for u, v, data in tqdm(G.edges(data = True), desc = '\t(05/10)'):
        w = data['weight'] if 'weight' in data else 1.0
        if H.has_edge(u, v):
            H[u][v]['weight'] += w
        else:
            H.add_edge(u, v, weight=w)

    nx.write_graphml_lxml(H, f'./graph/roblox-{model_option}.graphml')

    # print(list(H.nodes(data = True))[:10])
    # print(list(H.edges(data = True))[:10])

    # DATASET - ZEPETO
    df = pd.read_csv(f'./datasets/{model_option}/zepeto.csv', index_col = 0, low_memory = False)

    # PREPROCESS
    tqdm.pandas(desc = '\t(06/10)')
    df['keybert_keywords'] = df['keybert_keywords'].progress_apply(eval)

    tqdm.pandas(desc = '\t(07/10)')
    df['keywords'] = df['keybert_keywords'].progress_apply(lambda x : list(flatten(x))[0::2] if len(x) > 0 else '')

    count_df = df[['keywords', 'userName']].explode('keywords').groupby('keywords').count()

    count_df.columns = ['count']
    
    # GRAPH FORMULATION - ZEPETO
    G = nx.MultiGraph()
    unique_items = df['keywords'].explode().unique()

    # ADD NODES
    for item in tqdm(unique_items, desc = '\t(08/10)'):
        if count_df.loc[item]['count'] > np.percentile(count_df['count'], 99):
            G.add_node(item, keyword = item, weight = count_df.loc[item]['count'])
    
    selected = list(count_df[count_df['count'] > np.percentile(count_df['count'], 99)].index)
    selected = [word for word in selected if word.isalpha()]

    # ADD EDGES
    for keywords in tqdm(df['keywords'], desc = '\t(09/10)'):
        keyword_selected = [keyword for keyword in keywords if keyword in selected]

        pairs = list(combinations(keyword_selected, 2))
        pairs = [(pair[0], pair[1]) if pair[0] < pair[1] else (pair[1], pair[0]) for pair in pairs]
        
        G.add_edges_from(pairs)
    
    # MULTI to SINGLE - ZEPETO
    H = nx.Graph()
    unique_items = df['keywords'].explode().unique()

    H.add_nodes_from([node for node in G.nodes(data = True) if node[1]['keyword'] in selected])

    for u, v, data in tqdm(G.edges(data = True), desc = '\t(10/10)'):
        w = data['weight'] if 'weight' in data else 1.0
        if H.has_edge(u, v):
            H[u][v]['weight'] += w
        else:
            H.add_edge(u, v, weight=w) 
    nx.write_graphml_lxml(H, f'./graph/zepeto-{model_option}.graphml')

    # print(list(H.nodes(data = True))[:10])
    # print(list(H.edges(data = True))[:10])


[0/18] - keybert-all-MiniLM-L12-v2-post-tokenized


	(10/10): 100%|██████████| 753946/753946 [00:01<00:00, 527498.26it/s]


[1/18] - keybert-all-MiniLM-L12-v2-tokenized


	(10/10): 100%|██████████| 774065/774065 [00:01<00:00, 538653.68it/s]


[2/18] - keybert-all-MiniLM-L12-v2


...

	(10/10): 100%|██████████| 774065/774065 [00:01<00:00, 541600.44it/s]


[8/18] - keybert-all-mpnet-base-v2


	(10/10): 100%|██████████| 760106/760106 [00:01<00:00, 486570.37it/s]


[9/18] - keybert-distilbert-base-nli-mean-tokens-post-tokenized


	(04/10):  12%|█▏        | 53119/446700 [00:09<01:04, 6055.51it/s]